In [ ]:
##### Stress and strain for a hole in a plate #####
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
plt.ion()
plt.close('all')
mpl.rcParams.update({'font.size': 15})

## Define sample properties
E = 3.2e9 #Young's Modulus [N/m^2]
nu = 0.4  #Poisson's Ratio []
W = .03		#ROI Width [m]
H = .1		#ROI Height [m]
T = 0.00313 #Thickness[m]
Load = np.linspace(0,5000,6)
stress = Load/(W*T)	#Applied stress [N/m^2]
a = 0.003125		#Hole radius [m]
G = E/(2*(1+nu))

## Set Parameters
n = 1000 	#Spatial Resolution
p1 = [500,900] #Away in axial direction
p4 = [20,500]  #Away in transverse direction
p3 = [650,500] #1Sig concentration point
p2 = [500,620] #3Sig point

x = np.linspace(-W/2,W/2,n)	#x grid
y = np.linspace(-H/2,H/2,n)	#y grid
[X,Y] = np.meshgrid(x,y)
r = (X**2 + Y**2)**0.5		#r grid
th = np.arctan(Y/X) + np.pi/2		#theta grid
twoPi = np.linspace(0,2*np.pi,100)
xr = a*np.cos(twoPi)	#Hole x
yr = a*np.sin(twoPi)	#Hole y

axial = np.zeros((6,4))
transverse = np.zeros((6,4))
shear = np.zeros((6,4))

for i,so in enumerate(stress):
    ## Calculate Hoop and Radial Stresses
    srr = so/2*((1-a**2/r**2) + (1+3*a**4/r**4-4*a**2/r**2)*np.cos(2*th))
    srr[r<=a] = 0
    stt = so/2*((1+a**2/r**2) - (1+3*a**4/r**4)*np.cos(2*th))
    stt[r<=a] = 0
    srt =-so/2*(1 + 2*a**2/r**2 -3*a**4/r**4)*np.sin(2*th)
    srt[r<=a] = 0

    ## Convert to Cartesian Stress
    sxx = srr*np.cos(th)**2 + stt*np.sin(th)**2 - srt*np.sin(2*th)
    syy = srr*np.sin(th)**2 + stt*np.cos(th)**2 + srt*np.sin(2*th)
    sxy = np.sin(th)*np.cos(th)*(srr-stt) + srt*np.cos(2*th)

    ## Convert to Strain
    exx = (sxx - nu*syy)/E
    eyy = (syy - nu*sxx)/E
    exy = sxy/G

    axial[i,0] = exx[p1[0],p1[1]]*100
    axial[i,1] = exx[p2[0],p2[1]]*100
    axial[i,2] = exx[p3[0],p3[1]]*100
    axial[i,3] = exx[p4[0],p4[1]]*100

    transverse[i,0] = eyy[p1[0],p1[1]]*100
    transverse[i,1] = eyy[p2[0],p2[1]]*100
    transverse[i,2] = eyy[p3[0],p3[1]]*100
    transverse[i,3] = eyy[p4[0],p4[1]]*100

    shear[i,0] = exy[p1[0],p1[1]]*100
    shear[i,1] = exy[p2[0],p2[1]]*100
    shear[i,2] = exy[p3[0],p3[1]]*100
    shear[i,3] = exy[p4[0],p4[1]]*100
plt.figure(figsize=(8,7))
plt.plot(axial[:,0],Load,color = 'r',marker = 'o',label = 'Strain Gauge1')
plt.plot(axial[:,1],Load,color = 'g',marker = 'o',label = 'Strain Gauge2')
plt.plot(axial[:,2],Load,color = 'b',marker = 'o',label = 'Strain Gauge3')
plt.plot(axial[:,3],Load,color = 'aqua',marker = 'o',label = 'Strain Gauge4')
plt.xlabel('Axial Strain [%]')
plt.ylabel('Load [N]')
plt.legend()
plt.xlim([-2,12])
#%%
## Plot Results
Strains = [exx,eyy,exy]
Types = ['{xx}','{yy}','{xy}']
for e,t in zip(Strains,Types):
    fig = plt.figure(figsize=(8,7))
    ax = fig.gca()
    ax.plot(xr,yr,'k--')
    eC = ax.contourf(X,Y,e, vmin=-so/E, vmax=3*so/E, cmap='jet', levels=100)
    ax.set_aspect('equal', 'box')
    fig.colorbar(eC)
    ax.set_title('$\epsilon_'+t+'$')

Stress = [srr,stt,srt]
Types = ['{rr}','{θθ}','{rθ}']
for e,t in zip(Stress,Types):
    fig = plt.figure(figsize=(8,7))
    ax = fig.gca()
    ax.plot(xr,yr,'k--')
    eC = ax.contourf(X,Y,e, vmin=-so, vmax=3*so, cmap='jet', levels=12)
    ax.set_aspect('equal', 'box')
    fig.colorbar(eC)
    ax.set_title('$\sigma_'+t+'$')
    plt.tight_layout()